# SNF REIT Analysis - Exploratory Analysis

This notebook demonstrates how to use the SNF REIT Analysis platform for exploratory data analysis.

In [1]:
import pandas as pd
import plotly.express as px
from snf_reit_analysis.data.loaders import CMSDataLoader, BLSDataLoader, SECDataLoader
from snf_reit_analysis.config import config

## 1. Load CMS Nursing Home Data

In [2]:
# Initialize CMS data loader
cms = CMSDataLoader()

# Fetch provider information for high-rated facilities
providers = cms.get_provider_info(min_rating=4, use_polars=False)
print(f"Fetched {len(providers):,} provider records")
providers.head()

Fetched 14,752 provider records


,cms_certification_number_ccn,provider_name,provider_address,citytown,state,zip_code,telephone_number,provider_ssa_county_code,countyparish,ownership_type,...,number_of_citations_from_infection_control_inspections,number_of_fines,total_amount_of_fines_in_dollars,number_of_payment_denials,total_number_of_penalties,location,latitude,longitude,geocoding_footnote,processing_date
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,290,Franklin,For profit - Corporation,...,,1,23989.00,0,1,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",34.5149,-87.736,,2025-09-01
1,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,For profit - Corporation,...,0,0,0.00,0,0,"260 WEST WALNUT STREET,SYLACAUGA,AL,35150",33.1637,-86.254,,2025-09-01
2,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,Government - County,...,,0,0.00,0,0,"380 WOODS COVE ROAD,SCOTTSBORO,AL,35768",34.6611,-86.047,,2025-09-01
3,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,For profit - Individual,...,0,0,0.00,0,0,"7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206",33.5595,-86.722,,2025-09-01
4,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,360,Jefferson,For profit - Individual,...,,0,0.00,0,0,"6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111",33.3222,-87.034,,2025-09-01


## 2. Load BLS Economic Indicators

In [3]:
# Initialize BLS data loader
bls = BLSDataLoader()

# Fetch PPI data
ppi_data = bls.get_nursing_facility_ppi(start_year="2020", end_year="2024")
print(f"Fetched {len(ppi_data):,} PPI records")
ppi_data.head()

Fetched 195 PPI records


,series_id,series_name,year,period,period_name,value,latest
0,PCU623110623110,PPI Overall,2024,M13,Annual,277.772,False
1,PCU623110623110,PPI Overall,2024,M12,December,283.049,False
2,PCU623110623110,PPI Overall,2024,M11,November,281.094,False
3,PCU623110623110,PPI Overall,2024,M10,October,280.816,False
4,PCU623110623110,PPI Overall,2024,M09,September,277.284,False


## 3. Load SEC REIT Financials

In [4]:
# Initialize SEC data loader
sec = SECDataLoader()

# Fetch financial data for Omega Healthcare (OHI)
ohi_financials = sec.get_reit_financials("OHI")
print(f"Fetched {len(ohi_financials)} financial metrics for OHI")

# Display total assets
ohi_financials["Assets"].head()

Fetched 8 financial metrics for OHI


,end,val,accn,fy,fp,form,filed,frame
127,2025-06-30,10546476000,0000888491-25-000025,2025.0,Q2,10-Q,2025-08-01,CY2025Q2I
126,2025-03-31,9705738000,0000888491-25-000016,2025.0,Q1,10-Q,2025-05-02,CY2025Q1I
125,2024-12-31,9897891000,0000888491-25-000025,2025.0,Q2,10-Q,2025-08-01,CY2024Q4I
124,2024-12-31,9897891000,0000888491-25-000016,2025.0,Q1,10-Q,2025-05-02,NaN
123,2024-12-31,9897891000,0000888491-25-000006,2024.0,FY,10-K,2025-02-13,NaN


## 4. Visualization Examples

In [5]:
# PPI trends over time
fig = px.line(
    ppi_data,
    x="year",
    y="value",
    color="series_name",
    title="Nursing Facility PPI Trends"
)
fig.show()

In [6]:
# Rating distribution
if "Overall_Rating" in providers.columns:
    rating_counts = providers["Overall_Rating"].value_counts().sort_index()
    fig = px.bar(
        x=rating_counts.index,
        y=rating_counts.values,
        labels={"x": "Rating", "y": "Number of Facilities"},
        title="CMS 5-Star Rating Distribution"
    )
    fig.show()